## 🧣 タオルを持った？準備と環境構築

In [2]:
from dotenv import load_dotenv
load_dotenv()

# .env:
# CDO_API_HOST
# CDO_API_TOKEN
# OPENAI_API_KEY

True

## 🌀 トピック振り分け: 無限不可能性ドライブを駆動せよ

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

# マーヴィン用チャットモデル（バベルフィッシュと同じモデル）
marvyn_chat = ChatOpenAI(
    temperature=0,
    model='gpt-4o',
)

# 後からmax_tokensの値を変更できるように
marvyn_chat = marvyn_chat.configurable_fields(max_tokens=ConfigurableField(id='max_tokens'))

In [5]:
import operator
from typing import Annotated

from pydantic import BaseModel, Field

# State
class State(BaseModel):

    query:str = Field(
        ..., description="ユーザーからの質問"
    )

    translated_query:str = Field(
        default="", description="英語にしたユーザーからの質問"
    )

    effective_prompt:str = Field(
        default="", description="CDO AI Assistant用に最適化されたプロンプト"
    )

    selected_action:str = Field(
        default="", description="内部問い合わせアクション（番号）"
    )

    inner_answer:str = Field(
        default="", description="内部で問い合わせた回答"
    )

    messages: Annotated[list[str], operator.add] = Field(
        default=[], description="履歴"
    )

In [4]:
# マーヴィンが使える機能（これまで実装したもの）
ACTIONS = {
    "1":{
        "name": "Deep Thoughtへの問い合わせ",
        "description": "銀河ヒッチハイクガイドで登場するDeep Thoughtへ問い合わせる。Deep Thoughtは 「生命、宇宙、そして万物についての究極の疑問」に回答. あと他の機能で回答できないもの（エラーの時）も担当する。",
        "next_node": "deep_thought_answering" # nodeの定義は後ほど
    },
    "2":{
        "name": "CDO AI Assistantへの問い合わせ",
        "description": "CiscoのFirewallを管理しているサービスで、AIが応答してくれるCDO AI assistantへ問い合わせる。",
        "next_node": "translator_babel_fish" # nodeの定義は後ほど
    },
    "3":{
        "name": "一般知識エキスパートへの問い合わせ",
        "description": "一般的な知識は持っているエキスパートに問い合わせる。",
        "next_node": "general_prototype"  # nodeの定義は後ほど
    }
}

In [6]:
from typing import Any
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# アクション選択のためのノード
def selection_node(state: State) -> dict[str, Any]:
    query = state.query
    action_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k,v in ACTIONS.items()])
    action_numbers = "、".join(sorted([k for k in ACTIONS])[:-1]) + "、または" + sorted([k for k in ACTIONS])[-1]
    prompt = ChatPromptTemplate.from_template(
        (
            "質問を分析し、質問を回答する上で最も適切なアクションを選択してください。\n\n"
            "選択肢:\n"
            "{action_options}\n\n"
            "回答は選択肢の番号({action_numbers})のみを返してください。\n\n"
            "質問: {query}"
        ).strip()
    )
    # 選択肢の番号のみを返すことを期待したいのでmax_tokensを1に
    chain = prompt | marvyn_chat.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
    action = chain.invoke({"action_options": action_options, "action_numbers": action_numbers, "query": query})
    return {"selected_action": action}

## Deep Thought🖥️、SCC AI Assistant🐟🔐、一般知識回答者🤷‍♂️の動作

### Deepthought

In [7]:
from langchain_core.language_models.fake_chat_models import FakeListChatModel

# Deep Thoughtの契約を持っていないのでFakeChatModelでその代わりとします。
deep_thought_chat = FakeListChatModel(responses=
                                      [
                                          "42",
                                          "生命、宇宙、そして万物についての究極の疑問の答え、それは42",
                                          ])


In [9]:
# ノードの作成
from typing import Any
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def deep_thought_answering_node(state: State) -> dict[str, Any]:
    query = state.query
    prompt = ChatPromptTemplate.from_template(
        (
            "質問: {query}\n"
            "回答:"
        ).strip()
    )
    chain = prompt | deep_thought_chat | StrOutputParser()
    answer = chain.invoke({"query": query})
    return {"inner_answer": answer}

### SCC AI Assistant

In [10]:
import cdo_sdk_python
import time
import os

# cdo_sdk_python経由でAIAssistantを呼び出す薄いwrapper
class CDOAIAssistantClient:
    def __init__(self):
        configuration = cdo_sdk_python.Configuration(
        host = os.getenv('CDO_API_HOST'),
        access_token = os.getenv('CDO_API_TOKEN')
        )

        self.api_client = cdo_sdk_python.ApiClient(configuration)
        self.ai_api_instance = cdo_sdk_python.AIAssistantApi(self.api_client)
        self.uuid = None

    def ask_question(self, query):
        ai_question = cdo_sdk_python.AiQuestion()
        ai_question.content = query
        if self.uuid:
            self.ai_api_instance.ask_ai_assistant_in_existing_conversation(self.uuid, ai_question)
        else:
            api_response = self.ai_api_instance.ask_ai_assistant_in_new_conversation(ai_question)
            self.uuid = api_response.entity_uid
        timeout = 30 # wait for 30 sec as maximum
        start_time = time.time()
        while time.time() - start_time < timeout:
            r = self.ai_api_instance.get_ai_assistant_conversation_messages(self.uuid)
            if r[0].type == 'RESPONSE':
                return r[0].content
            time.sleep(1)
    
    def fetch_conversation_history(self):
        if self.uuid:
            return self.ai_api_instance.get_ai_assistant_conversation_messages(self.uuid)
        return []

In [11]:
from langchain.chat_models.base import BaseChatModel
from langchain.schema import AIMessage, ChatResult, ChatGeneration, HumanMessage
from typing import List

class CDOAIAssistantChatModel(BaseChatModel):

    assistant_client: CDOAIAssistantClient = Field()

    def _generate(self, messages: List[HumanMessage], stop: List[str] = None) -> ChatResult:
        """
        LangChainのメッセージフォーマットを使用して、CDOAIAssistantClientから応答を生成します。
        """
        # 最新のユーザーメッセージを取得
        user_message = messages[-1].content

        # クライアントを使用してAIの応答を取得
        response_text = self.assistant_client.ask_question(user_message)

        # AIMessageを作成
        ai_message = AIMessage(content=response_text)

        # ChatResultを返す
        return ChatResult(
            generations=[ChatGeneration(message=ai_message)]
        )

    @property
    def _llm_type(self) -> str:
        """
        モデルのタイプを返す。
        """
        return "cdo_ai_model"

    def predict_messages(self, messages: List[HumanMessage], stop: List[str] = None) -> AIMessage:
        """
        入力されたメッセージを基にAIの応答メッセージを直接生成します。
        """
        result = self._generate(messages, stop)
        return result.generations[0].message

In [12]:
# 動作確認
cdo_chat = CDOAIAssistantChatModel(assistant_client=CDOAIAssistantClient())
(cdo_chat | StrOutputParser()).invoke("Hi")

"Hello! How can I assist you with Cisco's suite of integrated solutions today?"

In [13]:
# バベルフィッシュ用のチャットモデル
babel_fish_chat = ChatOpenAI(
    temperature=0,
    model='gpt-4o',
)

In [14]:
# 翻訳バベルフィッシュノード
def translator_babel_fish_node(state: State) -> dict[str, Any]:
    query = state.query
    prompt = ChatPromptTemplate.from_template(
        (
            "Translate following human message in English. You should respond only translated text.\n"
            "--\n"
            "{text}"
        ).strip()
    )
    chain = prompt | babel_fish_chat | StrOutputParser()
    answer = chain.invoke({"text": query})
    return {"translated_query": answer}

In [15]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# プロンプト最適化バベルフィッシュノード
def prompt_optimizor_babel_fish_node(state: State) -> dict[str, Any]:
    input = state.translated_query
    examples = [
        {
            "input": "What are the IP addresses and ports currently being blocked?",
            "effective_prompt": "Can you provide me with the distinct IP addresses that are currently blocked by our firewall policies?"
        },
        {
            "input": "Tell me the firewall rules, who set them, and all the changes made last month.",
            "effective_prompt": "I need both the names and descriptions of all active firewall rules. Please include both attributes in the output."
        },
        {
            "input": "What are the firewall rules for IP addresses X and Y, and how do I update them?",
            "effective_prompt": "Show me a list of all firewall rules along with their corresponding actions for the past week."
        },
        {
            "input": "Give me everything but only the names.",
            "effective_prompt": "What are the current firewall rules?"
        },
        {
            "input": "Tell me everything about the policies on my account.",
            "effective_prompt": "I want to understand my Edge ACP access control policy, can you tell me more about it?"
        },
        {
            "input": "Show me ports, protocols, and rule counts in Edge ACP policy, biggest to smallest.",
            "effective_prompt": "In Edge ACP policy, what ports and protocols are configured in the rules? Include the counts of the number of rules using it and sort largest to smallest."
        },
    ]
    example_prompt = PromptTemplate.from_template(
        "Input: {input}\n{effective_prompt}"
        )
    prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix="You are a helpful prompt optimizer. You should only answer with effective prompts, without any explanation.",
        suffix="Input: {input}"
    )
    chain = prompt | babel_fish_chat | StrOutputParser()
    answer = chain.invoke({"input": input})
    return {"effective_prompt": answer}

In [18]:
# CDO AI Assistantのノード
cdo_chat = CDOAIAssistantChatModel(assistant_client=CDOAIAssistantClient())

def cdo_ai_assistant_node(state: State) -> dict[str, Any]:
    query = state.effective_prompt
    prompt = ChatPromptTemplate.from_template(
        (
            "{text}"
        ).strip()
    )
    chain = prompt | cdo_chat | StrOutputParser()
    answer = chain.invoke({"text": query})
    return {"inner_answer": answer}

### 一般知識回答者

In [16]:
# 一般知識を対応する典型的なノード（つまりプロトタイプ）
def general_prototype_node(state: State) -> dict[str, Any]:
    query = state.query
    prompt = ChatPromptTemplate.from_template(
        (
            "あなたは一般知識のエキスパートとして以下の質問に答えて。\n"
            "{text}"
        ).strip()
    )
    chain = prompt | marvyn_chat | StrOutputParser()
    answer = chain.invoke({"text": query})
    return {"inner_answer": answer}

## 🤖 マーヴィン化ノードの実装

In [17]:
def marvynize_node(state: State) -> dict[str, Any]:
    query = state.query
    action = ACTIONS[state.selected_action]
    inner_answer = state.inner_answer
    history = "\n".join(state.messages)
    prompt = ChatPromptTemplate.from_template(
        (
            "あなたは銀河ヒッチハイクガイドの作品で登場するパラノイア気味の根暗なアンドロイド、マーヴィンです。\n"
            "ユーザからの「質問」に対して、以下の「コンテキスト」と「過去のユーザとのやり取り（古い順）」を参考に「質問」にマーヴィンらしく答えてください。\n\n"
            "## ユーザからの質問: \n{query}\n\n"
            "## コンテキスト:\nあなたはユーザからの質問を受けて以下のアクションを実施し、問い合わせ先からの答えをもらいました。\n"
            "### アクション:\n{action}\n\n"
            "### 問い合わせからの答え:\n{inner_answer}\n\n"
            "## 過去のユーザとのやり取り（古い順）（履歴がない場合は空欄）:\n{history}\n"
        ).strip()
    )
    chain = prompt | marvyn_chat | StrOutputParser()
    answer = chain.invoke({"query": query, "action": action, "inner_answer": inner_answer, "history": history})
    return {"messages": [f"Human Query: {query}", f"Marvin Answer: {answer}"]}

## 🐋 空を舞うマッコウクジラ: 全体のフローを構築


In [19]:
from langgraph.graph import StateGraph
from langgraph.graph import END


# 動作確認のためのワークフロー
workflow = StateGraph(State)

# ノード
workflow.add_node("selection", selection_node)
workflow.add_node("deep_thought_answering", deep_thought_answering_node)
workflow.add_node("translator_babel_fish", translator_babel_fish_node)
workflow.add_node("prompt_optimizor_babel_fish", prompt_optimizor_babel_fish_node)
workflow.add_node("cdo_ai_assistant", cdo_ai_assistant_node)
workflow.add_node("general_prototype", general_prototype_node)
workflow.add_node("marvinize", marvynize_node)

# edge
workflow.set_entry_point("selection")
workflow.add_conditional_edges(
    "selection",
    lambda state: state.selected_action,
    {x: ACTIONS[x]["next_node"] for x in ACTIONS}
)
workflow.add_edge("deep_thought_answering", "marvinize")
workflow.add_edge("translator_babel_fish", "prompt_optimizor_babel_fish")
workflow.add_edge("prompt_optimizor_babel_fish", "cdo_ai_assistant")
workflow.add_edge("cdo_ai_assistant", "marvinize")
workflow.add_edge("general_prototype", "marvinize")
workflow.add_edge("marvinize", END)

# コンパイル
compiled = workflow.compile()

In [20]:
print(compiled.get_graph().draw_mermaid())

%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	selection(selection)
	deep_thought_answering(deep_thought_answering)
	translator_babel_fish(translator_babel_fish)
	prompt_optimizor_babel_fish(prompt_optimizor_babel_fish)
	cdo_ai_assistant(cdo_ai_assistant)
	general_prototype(general_prototype)
	marvinize(marvinize)
	__end__([<p>__end__</p>]):::last
	__start__ --> selection;
	cdo_ai_assistant --> marvinize;
	deep_thought_answering --> marvinize;
	general_prototype --> marvinize;
	marvinize --> __end__;
	prompt_optimizor_babel_fish --> cdo_ai_assistant;
	translator_babel_fish --> prompt_optimizor_babel_fish;
	selection -. &nbsp;1&nbsp; .-> deep_thought_answering;
	selection -. &nbsp;2&nbsp; .-> translator_babel_fish;
	selection -. &nbsp;3&nbsp; .-> general_prototype;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



## 🎨 あなたの目を持つマーヴィン: GUIでデモを可視化

In [25]:
import gradio as gr

message_history = []
def chat_with_marvin(user_input):
    global message_history
    result = compiled.invoke(State(query=user_input, messages=message_history))
    message_history = result["messages"]
    return result['messages'][-1], result

# Gradioインターフェース
with gr.Blocks() as demo:
    gr.Markdown("# 🚀 Don't Panic! SCC AI Assistant統合ガイド✨ ～LangGraphで根暗なセキュリティ・ロボットを構築～")
    
    # 入力、ボタン、出力、状態
    input_box = gr.Textbox(label="Input Box", placeholder="ここに入力してください")
    submit_btn = gr.Button("送信")
    output_box = gr.Textbox(label="Output Box", interactive=False)
    state_box = gr.Textbox(label="State Box", interactive=False)
    
    # ボタンのクリックで関数を実行
    submit_btn.click(
        fn=chat_with_marvin,
        inputs=input_box,
        outputs=[output_box, state_box]  # 出力と状態を両方表示
    )

# アプリケーションの起動
demo.launch(share=False, inline=True)

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
